In [5]:
def explore_alignments(i, j, current_align1, current_align2, alignments, score_matrix, seq1, seq2):
    # Condition d'arrêt
    if i < 2 and j < 2:
        alignments.append((current_align1, current_align2))
        return

    diag = score_matrix[i - 1][j - 1] if i > 0 and j > 0 else float('-inf')
    up = score_matrix[i - 1][j] if i > 0 else float('-inf')
    left = score_matrix[i][j - 1] if j > 0 else float('-inf')

    max_value = max(diag, up, left)

    if max_value == diag:
        explore_alignments(i - 1, j - 1, (seq1[j - 2] if i > 1 else '-') + current_align1, 
                           (seq2[i - 2] if j > 1 else '-') + current_align2, alignments, score_matrix, seq1, seq2)
    if max_value == up:
        explore_alignments(i - 1, j,  '-'+ current_align1, seq2[i - 2] + current_align2, alignments, score_matrix, seq1, seq2)
    if max_value == left:
        explore_alignments(i, j - 1, seq1[j - 2] + current_align1, '-' + current_align2, alignments, score_matrix, seq1, seq2)

    return alignments

In [6]:
#global alignement 
def global_al(seq1,seq2, match=2, mismatch=-1, gap=-2):
    match = 2
    mismatch=-1
    gap=-2

    col = len(seq1) + 1
    ligne = len(seq2) + 1

    score_matrix = [[0] * col for _ in range(ligne)]

    for i in range(1, ligne):
        score_matrix[i][0] = score_matrix[i - 1][0] + gap

    for j in range(1, col):
        score_matrix[0][j] = score_matrix[0][j - 1] + gap

    print(seq1,seq2)    

    for i in range(1, ligne):
        for j in range(1, col):
            if seq2[i - 1] == seq1[j - 1]:
                score = match
            else:
                score = mismatch    

            diag = score_matrix[i - 1][j - 1] + score
            up = score_matrix[i - 1][j] + gap
            left = score_matrix[i][j - 1] + gap
            max_score = max(diag, up, left)
            score_matrix[i][j] = max_score


    matrix_final_score = score_matrix[ligne-1][col-1]
    alignments = []  
    i, j = ligne - 1, col - 1
    initial_align1 = seq1[j-1]  
    initial_align2 = seq2[i-1] 

    alignments = explore_alignments(i, j, initial_align1, initial_align2,alignments,score_matrix,seq1,seq2)

    #for i in (score_matrix):
        #print(i)

    return alignments , score_matrix , matrix_final_score   


In [7]:
def local(seq1,seq2, match=2, mismatch=-1, gap=-2):
    ligne = len(seq2)+1
    col = len(seq1)+1

    print(seq1,seq2)

    score_matrix_loc =  [[0] * col for _ in range(ligne)]

    for i in range(1,ligne):

        for j in range(1,col):

            if seq2[i-1] == seq1[j-1]:
                score = match
            else :
                score = mismatch 

            diag = score_matrix_loc[i-1][j-1] + score
            up = score_matrix_loc[i-1][j] + gap 
            left = score_matrix_loc[i][j-1] + gap   
            score_matrix_loc[i][j] = max(0,diag,up,left)


    alignements_loc = [] 
    i, j  = ligne-1,col-1
    end = score_matrix_loc[i][j]
    diag = score_matrix_loc[i-1][j-1]
    up = score_matrix_loc[i-1][j]
    left = score_matrix_loc[i][j-1]  
    max_score_loc = max(diag,up,left,end)

    if diag == max_score_loc :

        print('diag')
        alignements_loc = explore_alignments(i-1, j-1, seq1[j-2], seq2[i-2],alignements_loc,score_matrix_loc,seq1,seq2)


    elif end == max_score_loc :
        print('end')
        alignements_loc = explore_alignments(i, j, seq1[j-1], seq2[i-1],alignements_loc,score_matrix_loc,seq1,seq2)    

    elif up == max_score_loc :
        print('up')
        alignements_loc = explore_alignments(i-1, j, seq1[j-1], seq2[i-2],alignements_loc,score_matrix_loc,seq1,seq2)

    elif left == max_score_loc :
        print('left')
        alignements_loc = explore_alignments(i, j-1, seq1[j-2], seq2[i-1],alignements_loc,score_matrix_loc,seq1,seq2)

    #for i in score_matrix_loc:
        #print(i)    

    return alignements_loc, score_matrix_loc, max_score_loc           

In [8]:
def calculate_score(align1, align2, match=2, mismatch=-1, gap=-2):
    score = 0
    for a, b in zip(align1, align2):
        if a == '-' or b == '-':
            score += gap
        elif a == b:
            score += match
        else:
            score += mismatch
    return score

In [9]:
def pair_par_pair(sequences,choose='global'):

    n = len(sequences)

    alignement_multiple = []
    
    for i in range(n):
        for j in range(i + 1, n):
            if choose == 'global':
                alignments, score_matrix_loc, max_score_loc = global_al(sequences[i],sequences[j])
            else :
                alignments, score_matrix_loc, max_score_loc = local(sequences[i],sequences[j])

            alignments_with_scores = []

            for align1 ,align2 in alignments:
                score_alignement = calculate_score(align1, align2)
                alignments_with_scores.append([align1, align2, score_alignement])
                alignments_with_scores.sort(key=lambda x: x[2],reverse=False)
            alignement_multiple.append(alignments_with_scores)

    return  alignement_multiple       
            

In [24]:
seq1 = 'ACCGAGA'
seq2 = 'ACGTCCCT'

print(calculate_score(seq1, seq2))
alignments, score_matrix_loc, max_score_loc = local(seq1,seq2)
print(max_score_loc)
scored_alignments = [(align1, align2, f'score = {calculate_score(align1, align2,2,-1,-1)}' ) for align1, align2 in alignments]
print(scored_alignments)

-1
ACCGAGA ACGTCCCT
diag
0
[('--AACCGAG', 'AC-TCC--C', 'score = -3'), ('-A-ACCGAG', 'A-GTCC--C', 'score = -3'), ('---ACCGAG', 'ACGTCC--C', 'score = -3'), ('--A-ACCGAG', 'AC-GTCC--C', 'score = -4'), ('---AACCGAG', 'ACG-TCC--C', 'score = -4'), ('A---CCGAG', 'ACGTCC--C', 'score = 0'), ('---AACCGAG', 'ACG--CC--C', 'score = -4'), ('--A-ACCGAG', 'AC-T-CC--C', 'score = -4'), ('-A--ACCGAG', 'A-GT-CC--C', 'score = -4'), ('----ACCGAG', 'ACGT-CC--C', 'score = -4'), ('--A--ACCGAG', 'AC-GT-CC--C', 'score = -5'), ('---A-ACCGAG', 'ACG-T-CC--C', 'score = -5'), ('----AACCGAG', 'ACGT--CC--C', 'score = -5'), ('--AAC-CGAG', 'AC-T-CC--C', 'score = -7'), ('-A-AC-CGAG', 'A-GT-CC--C', 'score = -7'), ('---AC-CGAG', 'ACGT-CC--C', 'score = -7'), ('--A-AC-CGAG', 'AC-GT-CC--C', 'score = -8'), ('---AAC-CGAG', 'ACG-T-CC--C', 'score = -8'), ('A---C-CGAG', 'ACGT-CC--C', 'score = -4'), ('---AAC-CGAG', 'ACG---CC--C', 'score = -8'), ('--A-AC-CGAG', 'AC-T--CC--C', 'score = -8'), ('-A--AC-CGAG', 'A-GT--CC--C', 'score = -8'

In [11]:
sequences = [seq1,seq2,'ACGATCCT']
alignement  = pair_par_pair(sequences,'l')
print(alignement)

ACCGAGAC ACGTCCCT
up
ACCGAGAC ACGATCCT
up
ACGTCCCT ACGATCCT
end
[[['ACCGAGA--C', 'AC---GTCCC', -3]], [['ACCGAGA-C', 'AC-GAT-CC', 3]], [['ACG-TCCCT', 'ACGATCC-T', 10]]]


In [12]:
import numpy as np

In [13]:
def delete_indel(seq):
    return seq.replace('-','')

In [14]:
def calculate_distance(seq1, seq2):
    min_len = min(len(seq1), len(seq2))
    differences = sum(1 for a, b in zip(seq1[:min_len], seq2[:min_len]) if a != b)
    return differences

In [15]:
def compute_distance_matrix(sequences):

    n = len(sequences)
    distance_matrix = [[0] * n for _ in range(n)]

    for i in range(n):
        for j in range(i + 1, n):
            distance = calculate_distance(sequences[i], sequences[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance

    print("Matrice des distances :")
    for row in distance_matrix:
        print(row)
        
    return distance_matrix

def print_distance_matrix(matrix, labels):
    print("\nMatrice de distances:")
    print("   ", "  ".join(labels))
    for i, row in enumerate(matrix):
        print(f"{labels[i]:<3} {row}")

In [16]:
def upgma(matrix, data_a, data_b, new_label, labels):
    
    new_matrix = []
    new_labels = [label for i, label in enumerate(labels) if i not in (data_a, data_b)]
    new_labels.append(new_label)
    
    # Nvelle matrice de distances
    for i in range(len(matrix)):
        if i in (data_a, data_b):
            continue
        new_row = []
        for j in range(len(matrix)):
            if j in (data_a, data_b):
                continue
            new_row.append(matrix[i][j])
        new_matrix.append(new_row)
    
    # Calcul des distanceclusters
    new_data_distances = []

    for i in range(len(matrix)):
        if i not in (data_a, data_b):
            dist = (matrix[i][data_a] + matrix[i][data_b]) / 2
            new_data_distances.append(dist)
    new_data_distances.append(0)  # lui-même est 0
    
    for i, row in enumerate(new_matrix):
        row.append(new_data_distances[i])    
    new_matrix.append(new_data_distances)
    
    return new_matrix, new_labels

In [17]:
def upgma_processing(sequences):
    
    dist_matrix = compute_distance_matrix(sequences)
    labels = [f"I{i+1}" for i in range(len(sequences))]
    step = 1

    branch_distances = {}  
    
    while len(dist_matrix) > 1:

        min_dist = float("inf")
        cluster_a, cluster_b = -1, -1
        for i in range(len(dist_matrix)):
            for j in range(i + 1, len(dist_matrix)):
                if dist_matrix[i][j] < min_dist:
                    min_dist = dist_matrix[i][j]
                    cluster_a, cluster_b = i, j
        
        
        print(f"\nÉtape {step} : Regrouper {labels[cluster_a]} et {labels[cluster_b]} (distance = {min_dist})")
        new_label = f"({labels[cluster_a]},{labels[cluster_b]})"
        
        branch_distance = min_dist / 2
        branch_distances[new_label] = branch_distance

        print(f"Distance de branche pour {new_label} = {branch_distance}")
        
        dist_matrix, labels = upgma(dist_matrix, cluster_a, cluster_b, new_label, labels)
        print_distance_matrix(dist_matrix, labels)
        step += 1

    print("\nClustering terminé!")
    print(labels[0])
    print("\nDistances des branches :")
    for cluster, dist in branch_distances.items():
        print(f"{cluster} : {dist}")

In [18]:
# R_i 
def calculate_Ri(matrix):
    n = len(matrix)
    Ri = [sum(i) / (n - 2) for i in matrix] 
    return Ri

# S_ij = dij - ri -rj
def calculate_Sij(matrix, Ri):
    n = len(matrix)
    Sij = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                Sij[i][j] = matrix[i][j] - Ri[i] - Ri[j]
    return Sij

In [19]:
# Maj 
def NJ(matrix, cluster_a, cluster_b):

    n = len(matrix)
    new_matrix = []

    for i in range(n):
        if i != cluster_a and i != cluster_b:
            new_row = []
            for j in range(n):
                if j != cluster_a and j != cluster_b:
                    new_row.append(matrix[i][j])
            new_matrix.append(new_row)

    # nvl distances
    new_cluster_distances = []
    for i in range(n):
        if i not in (cluster_a, cluster_b):
            dist = (matrix[i][cluster_a] + matrix[i][cluster_b]-matrix[cluster_a][cluster_b]) / 2
            new_cluster_distances.append(dist)
    new_cluster_distances.append(0)  # Distance à lui-même

    for i, row in enumerate(new_matrix):
        row.append(new_cluster_distances[i])
    new_matrix.append(new_cluster_distances)
    
    return new_matrix

In [20]:
def NJ_processing(sequences):
    dist_matrix = compute_distance_matrix(sequences)
    labels = [f"I{i+1}" for i in range(len(sequences))]
    step = 1
    while len(dist_matrix) > 1:

        print(f"\n=== Étape {step} ===")
        print_distance_matrix(dist_matrix, labels)

        if len(dist_matrix) > 2:    

            # Calcul de R_i
            Ri = calculate_Ri(dist_matrix)
            print(f"\nR_i = {Ri}")
            
            # Calcul de S_ij
            Sij = calculate_Sij(dist_matrix, Ri)
            print("\nS_ij =")
            print(Sij)
            
            # data proche(min Sij)
            min_value = float("inf")
            cluster_a, cluster_b = -1, -1
            for i in range(len(Sij)):
                for j in range(i + 1, len(Sij)):
                    if Sij[i][j] < min_value:
                        min_value = Sij[i][j]
                        cluster_a, cluster_b = i, j

        if len(dist_matrix) == 2:
            cluster_a, cluster_b = 0, 1
            min_value = None

        print(f"Regroupement des data {labels[cluster_a]} et {labels[cluster_b]} (S_ij = {min_value})")
        
        if len(dist_matrix) > 2:
            
            # Calcul des nouvelles distances
            d_a = (dist_matrix[cluster_a][cluster_b] + Ri[cluster_a] - Ri[cluster_b]) / 2
            d_b = (dist_matrix[cluster_a][cluster_b] + Ri[cluster_b] - Ri[cluster_a]) / 2

            print(f"Distance {labels[cluster_a]} -> arbre = {d_a}")
            print(f"Distance {labels[cluster_b]} -> arbre = {d_b}")
        
        new_label = f"({labels[cluster_a]},{labels[cluster_b]})"
        labels = [label for i, label in enumerate(labels) if i not in (cluster_a, cluster_b)] + [new_label]
        dist_matrix = NJ(dist_matrix, cluster_a, cluster_b)
        
        step += 1

    print("\nClustering terminé!")
    print(labels[0])

In [21]:
NJ_processing(sequences)

Matrice des distances :
[0, 6, 6]
[6, 0, 2]
[6, 2, 0]

=== Étape 1 ===

Matrice de distances:
    I1  I2  I3
I1  [0, 6, 6]
I2  [6, 0, 2]
I3  [6, 2, 0]

R_i = [12.0, 8.0, 8.0]

S_ij =
[[0, -14.0, -14.0], [-14.0, 0, -14.0], [-14.0, -14.0, 0]]
Regroupement des data I1 et I2 (S_ij = -14.0)
Distance I1 -> arbre = 5.0
Distance I2 -> arbre = 1.0

=== Étape 2 ===

Matrice de distances:
    I3  (I1,I2)
I3  [0, 1.0]
(I1,I2) [1.0, 0]
Regroupement des data I3 et (I1,I2) (S_ij = None)

Clustering terminé!
(I3,(I1,I2))


In [22]:
mutat = "".join('choices')
print(mutat)

choices


In [ ]:
# Étape 1 : Calculer la distance entre deux séquences (Hamming distance)
def hamming_distance(seq1, seq2):
    length = max(len(seq1), len(seq2))
    seq1 = seq1.ljust(length, "-")
    seq2 = seq2.ljust(length, "-")
    return sum(base1 != base2 for base1, base2 in zip(seq1, seq2))

# Étape 2 : Calculer la matrice de distance à partir des séquences
def compute_distance_matrix(sequences):
    names = [f"Seq_{i+1}" for i in range(len(sequences))]
    n = len(sequences)
    distance_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i):  # Calculer uniquement pour j < i
            seq1 = sequences[i]
            seq2 = sequences[j]
            distance_matrix[i, j] = hamming_distance(seq1, seq2)
            distance_matrix[j, i] = distance_matrix[i, j]  # Assurer la symétrie

    return names, distance_matrix

In [ ]:
from itertools import combinations

# Fonction pour calculer la distance (nombre de substitutions) entre deux séquences
def hamming_distance(seq1, seq2):
    return sum(c1 != c2 for c1, c2 in zip(seq1, seq2))

# Fonction récursive pour construire l'arbre
def maximum_parcimonie(sequences):
    # Si seulement une séquence, on retourne directement
    if len(sequences) == 1:
        return sequences[0], 0  # Aucun coût

    min_cost = float('inf')  # Initialise le coût minimum
    best_tree = None         # Initialise le meilleur arbre

    # Combinaisons pour diviser les séquences en deux sous-ensembles
    for subset in combinations(sequences, len(sequences) // 2):
        left_group = list(subset)
        right_group = [s for s in sequences if s not in left_group]

        # Appels récursifs pour les sous-groupes gauche et droit
        left_tree, left_cost = maximum_parcimonie(left_group)
        right_tree, right_cost = maximum_parcimonie(right_group)

        # Calculer le coût entre les deux sous-arbres
        cost_between = hamming_distance(left_tree, right_tree)

        # Coût total pour cet arrangement
        total_cost = left_cost + right_cost + cost_between

        # Mettre à jour le meilleur arbre si le coût est minimal
        if total_cost < min_cost:
            min_cost = total_cost
            best_tree = (left_tree, right_tree)

    return best_tree, min_cost

# Exemple d'utilisation
sequences = ["AAAC", "AAAA", "AGCA", "AACA"]  # Exemple de séquences
optimal_tree, optimal_cost = maximum_parcimonie(sequences)

print("Arbre optimal:", optimal_tree)
print("Coût minimal:", optimal_cost)
